In [82]:
import time
import itertools
import string
import random
import Levenshtein
import networkx as nx
from scipy.sparse import coo_matrix
import rpy2.robjects as robjects
from sys import argv
from math import factorial
random.seed(1)
start_time = time.clock()

minimum_aa = 4
maximum_aa = 20
min_seqs = 0
max_seqs = 500000

In [83]:
# this is the original function -- not used anymore
def get_matrix_elements(string_list):
    levenshtein_list = []
    rows = []
    cols = []

    for a, b in itertools.combinations(string_list, 2):
        levenshtein_distance = Levenshtein.distance(a, b)
        if levenshtein_distance < 2:
            levenshtein_list.append(1)
            rows.append(d_i[a])
            cols.append(d_i[b])
    return levenshtein_list, rows, cols

In [84]:
print 'Number of iterations will be %d'%(max_seqs*(max_seqs-1)/2)

Number of iterations will be 124999750000


In [126]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
import os, sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['SPARK_DRIVER_MEMORY'] = '1500m'

conf = SparkConf()

In [127]:
# define spark runtime parameters
number_of_executors = 100
cores_per_executor = 4

In [128]:
conf.set('spark.executor.instances',  number_of_executors)
conf.set('spark.executor.cores', cores_per_executor)
conf.set('spark.executor.memory', '9g')

In [129]:
sc = SparkContext(master='yarn-client', conf=conf)

In [130]:
# generate the string list and lookup dictionaries (including reverse lookup)

string_list = []

for i in range(min_seqs, max_seqs):
    random_integer = random.randrange(minimum_aa, maximum_aa)
    s = ''.join(random.choice('ACDEFGHIKLMNPQRSTVWY') for _ in range(random_integer))
    string_list.append(s)

string_map = {}
string_map_inverse = {}
for i, s in enumerate(string_list):
    string_map[i] = s
    string_map_inverse[s] = i

In [131]:
def balance_partitions(x, N_partitions, N_elements): 
    """Balances out the number of elements across the partitions"""
    n_per_part = N_elements/N_partitions/2
    
    part = 0
    if x > N_elements/2:
        x = N_elements-x
    part = x/n_per_part
    return part if part < N_partitions else part-1

In [132]:
number_of_partitions = number_of_executors*cores_per_executor*10

In [142]:
# create an RDD of indices and balance partitioning
idx_rdd = sc.parallelize(xrange(max_seqs))\
            .map(lambda x: (x,None))\
            .partitionBy(number_of_partitions, lambda x: balance_partitions(x, number_of_partitions, max_seqs)).keys()

In [134]:
# broadcast the lookup dictionaries
string_map_b = sc.broadcast(string_map)
string_map_inverse_b = sc.broadcast(string_map_inverse)

In [135]:
def generate_matrix_elements_idx(iterator):
    """
    Generates non-zero matrix elements by calculating the Levenshtein
    distance for each string in the partition against all the strings in the 
    lookup dictionary. It only searches the strings with 0 < idx < my_idx, 
    i.e. only filling in the upper triangle of the matrix.
    """    
    
    lookup = string_map_b.value
    
    for my_idx in iterator: 
        for idx in range(my_idx):
            ld = Levenshtein.distance(lookup[my_idx], lookup[idx])
            if ld < 2 and ld > 0: 
                yield (my_idx, idx, float(ld))

In [137]:
import numpy as  np
from scipy import sparse

In [143]:
%%time
mat = idx_rdd.mapPartitions(generate_matrix_elements_idx)
mat_data = np.array(mat.collect())
distance_matrix = sparse.csr_matrix((mat_data[:,2], (mat_data[:,0], mat_data[:,1])), (max_seqs, max_seqs))

CPU times: user 1.48 s, sys: 221 ms, total: 1.7 s
Wall time: 8min 40s


In [144]:
# make the graph
import networkx

g = networkx.Graph(distance_matrix)